In [1]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import cv2
import csv
import mahotas

In [2]:
def color_moments_extraction(image):
        R = image[:,:,0]
        G = image[:,:,1]
        B = image[:,:,2]
        colorFeature=[
                np.mean(R), np.std(R),
                np.mean(G), np.std(G),
                np.mean(B), np.std(B)
        ]
        colorFeature = colorFeature/np.mean(colorFeature)
        return colorFeature.tolist()

In [3]:
def shape_extraction(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # extraction de la forme
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature.tolist()

In [4]:
def texture_extraction(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(np.uint8(image), cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    features = mahotas.features.haralick(image.astype(np.uint8)).ravel()
    return features

In [5]:
def HSV_extraction(image, mask=None):
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([np.uint8(image)], [0, 1, 2], mask, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    return hist.flatten()

In [6]:
features = np.empty([200,694], dtype=float)
i=0

for file in glob.glob("2Classes/*.jpg"):        
    image = plt.imread(file)

    #Apply features extraction
    f_color_moments = color_moments_extraction(image)
    f_shape = shape_extraction(image)
    f_texture = texture_extraction(image)
    f_Hsv = HSV_extraction(image)  
    
    #Concatenate features
    feature = np.hstack((f_color_moments, f_shape, f_texture, f_Hsv))
    
    features[i,...] = feature
    i += 1

## kmeans

In [7]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2,init = 'k-means++')
x = kmeans.fit_predict(features)

output = open("2C_prediction_kmeans.csv", "w")
i=0
for imagePath in glob.glob("2Classes/*.jpg"):
    imageID = imagePath.split("/")[-1]
    output.write("%s,%s\n" % (imageID, str(x[i]+1)))
    i += 1

output.close()

## gmm

In [8]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=2)
gmm.fit(features)
y = gmm.predict(features)

output = open("2C_prediction_gmm.csv", "w")
i=0
for imagePath in glob.glob("2Classes/*.jpg"):
    imageID = imagePath.split("/")[-1]
    output.write("%s,%s\n" % (imageID, str(y[i]+1)))
    i += 1

output.close()